# Better Predictions

## Reduce Model Variance with Ensemble Learning

## High Variance of MLP Model

In [1]:
# demonstrate high variance of mlp model on blobs classification problem
from sklearn.datasets import make_blobs
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from numpy import mean
from numpy import std
from matplotlib import pyplot
# fit and evaluate a neural net model on the dataset
def evaluate_model(trainX, trainy, testX, testy):
  # define model
  model = Sequential()
  model.add(Dense(15, input_dim=2, activation='relu'))
  model.add(Dense(3, activation='softmax')) 
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
  # fit model
  model.fit(trainX, trainy, epochs=200, verbose=0)
  # evaluate the model
  _, test_acc = model.evaluate(testX, testy, verbose=0)
  return test_acc
# generate 2d classification dataset
X, y = make_blobs(n_samples=500, centers=3, n_features=2, cluster_std=2, random_state=2)
y = to_categorical(y)
# split into train and test
n_train = int(0.3 * X.shape[0])
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]
# repeated evaluation
n_repeats = 30
scores = list()
for _ in range(n_repeats):
  score = evaluate_model(trainX, trainy, testX, testy) 
  print('> %.3f' % score)
  scores.append(score)
# summarize the distribution of scores
print('Scores Mean: %.3f, Standard Deviation: %.3f' % (mean(scores), std(scores))) # histogram of distribution
pyplot.hist(scores, bins=10)
pyplot.show()
# boxplot of distribution
pyplot.boxplot(scores)
pyplot.show()